In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)

%matplotlib inline

Using Device: cpu


In [2]:
# CIFAR-10 데이터셋 로드
# -----------------------
transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)
print('CIFAR-10 data ready!')

100%|██████████| 170M/170M [00:03<00:00, 43.1MB/s]


CIFAR-10 data ready!


In [14]:
# 5.2 ResNeXt Bottleneck
# -----------------------
def conv1x1(in_channels, out_channels, stride=1):
  return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

class ResNeXtBottleneck(nn.Module):
  expansion = 4
  def __init__(self, in_channels, out_channels, cardinality=8, base_width=4, stride=1, downsample=None):
    super(ResNeXtBottleneck, self).__init__()
    # width calculation
    width = int((out_channels // self.expansion) * (base_width / 64.0) * cardinality)
    # 1x1 reduce
    self.conv1 = nn.Conv2d(in_channels, width, kernel_size=1, bias=False)
    self.bn1 = nn.BatchNorm2d(width)
    # 3x3 group conv
    self.conv2 = nn.Conv2d(width, width, kernel_size=3,
    stride=stride, padding=1,
    groups=cardinality, bias=False)
    self.bn2 = nn.BatchNorm2d(width)
    # 1x1 expand
    self.conv3 = nn.Conv2d(width, out_channels, kernel_size=1, bias=False)
    self.bn3 = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU(inplace=True)
    self.downsample = downsample
    self.stride = stride

  def forward(self, x):
    identity = x

    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn3(out)

    if self.downsample is not None:
      identity = self.downsample(x)

    out += identity
    out = self.relu(out)
    return out

In [15]:
# 5.3 Simple ResNeXt for CIFAR-10
# -----------------------
class ResNeXtCIFAR(nn.Module):
  def __init__(self, layers=[2,2,2], cardinality=8, base_width=4, num_classes=10):
    super(ResNeXtCIFAR, self).__init__()
    self.in_channels = 64
    # initial conv
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace=True)
    # stages
    self.layer1 = self._make_layer(64, layers[0], cardinality, base_width, stride=1)
    self.layer2 = self._make_layer(128, layers[1], cardinality, base_width, stride=2)
    self.layer3 = self._make_layer(256, layers[2], cardinality, base_width, stride=2)
    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(256*ResNeXtBottleneck.expansion, num_classes)

  def _make_layer(self, out_channels, blocks, cardinality, base_width, stride=1):
    downsample = None
    if stride != 1 or self.in_channels != out_channels*ResNeXtBottleneck.expansion:
      downsample = nn.Sequential(
          conv1x1(self.in_channels, out_channels*ResNeXtBottleneck.expansion, stride),
          nn.BatchNorm2d(out_channels*ResNeXtBottleneck.expansion)
          )

    layers = []
    layers.append(
        ResNeXtBottleneck(
          self.in_channels,
          out_channels*ResNeXtBottleneck.expansion,
          cardinality=cardinality,
          base_width=base_width,
          stride=stride,
          downsample=downsample
        )
      )
    self.in_channels = out_channels*ResNeXtBottleneck.expansion

    for _ in range(1, blocks):
      layers.append(
        ResNeXtBottleneck(
        self.in_channels,
        out_channels*ResNeXtBottleneck.expansion,
        cardinality=cardinality,
        base_width=base_width
        )
    )
    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x

def resnext_cifar_demo():
   return ResNeXtCIFAR(layers=[2,2,2], cardinality=8, base_width=4, num_classes=10)

model = resnext_cifar_demo().to(device)
print(model)

ResNeXtCIFAR(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResNeXtBottleneck(
      (conv1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum